In [1]:

# Divides a set on a specific column. Can handle numeric or nominal values
#pseudocode for forming individual trees
def divideset(rows,column,value):
   # Make a function that tells us if a row is in the first group (true) or the second group (false)
   split_function=None
   if isinstance(value,int) or isinstance(value,float): # check if the value is a number i.e int or float
      split_function=lambda row:row[column]>=value
   else:
      split_function=lambda row:row[column]==value
   
   # Divide the rows into two sets and return them
   set1=[row for row in rows if split_function(row)]
   set2=[row for row in rows if not split_function(row)]
   return (set1,set2)


In [43]:
import numpy as np
import pandas as pd # Data frames
import matplotlib.pyplot as plt # Visuals
from sklearn.model_selection import train_test_split # Create training and test sets
from sklearn.neighbors import KNeighborsClassifier # Kth Nearest Neighbor
from sklearn.tree import DecisionTreeClassifier # Decision Trees
from sklearn.tree import export_graphviz # Extract Decision Tree visual
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.neural_network import MLPClassifier # Neural Networks
from sklearn.metrics import roc_curve # ROC Curves
from sklearn.metrics import auc # AUC 
from urllib.request import urlopen # Get data from UCI Machine Learning Repository
plt.style.use('ggplot')

Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
Hungarian_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
Switzerland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data'

np.set_printoptions(threshold=np.nan)#makes it so that we can see a whole array when we output it

names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heart disease(by angiography)']

ClevelandHeartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)#gets Cleveland data
HungarianHeartDisease = pd.read_csv(urlopen(Hungarian_data_URL), names = names)#gets Hungary data
SwitzerlandHeartDisease = pd.read_csv(urlopen(Switzerland_data_URL), names = names)#gets Switzerland data
datatemp = [ClevelandHeartDisease, HungarianHeartDisease, SwitzerlandHeartDisease]#combines all arrays into a list

#heartDisease.set_index(['id_number'], inplace = True)
#I don't know what that did, but it was from Raoul's code and it was giving me bugs so I got rid of it
data = pd.concat(datatemp)#combines list into one array
del data['ca']
del data['slope']
del data['thal']#these delete the columns that have a lot of ? values and so aren't very useful to us

data = data.replace('?',np.nan)#convert ? values to np.nan

for item in data:#converts everything to floats
    data[item] = pd.to_numeric(data[item])

def normalize(heartDisease, toNormalize):#normalizes 
    result = heartDisease.copy()
    for item in heartDisease.columns:
        if (item in toNormalize):
            max_value = heartDisease[item].max()
            min_value = heartDisease[item].min()
            result[item] = (heartDisease[item] - min_value) / (max_value - min_value)
    return result
toNormalize = ['age', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak']#columns to normalize
data = normalize(data, toNormalize)
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,heart disease(by angiography)
0,0.000000,1,0.333333,0.416667,0.218905,0.0,2.0,0.961538,0.0,0.342105,0
1,0.021739,1,0.333333,0.333333,0.402985,0.0,0.0,0.769231,0.0,0.342105,0
2,0.021739,1,0.333333,0.500000,NaN,0.0,0.0,0.846154,0.0,0.342105,0
3,0.043478,0,0.000000,0.750000,0.393035,0.0,1.0,0.846154,0.0,0.342105,0
4,0.065217,0,0.333333,0.166667,0.363184,0.0,1.0,0.692308,0.0,0.342105,0


In [26]:
#count = 0
#for item in data:
 #   for i in data[item]:
  #      count += (i=='?')
   # print(count)
    #count = 0


0
0
0
4
46
91
3
3
3
6
397
700
584
0


SyntaxError: invalid syntax (<ipython-input-4-f209e05a3ce8>, line 7)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heart disease(by angiography)
0,28,1,2,130,132,0,2,185,0,0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0,?,?,?,0


AttributeError: 'Series' object has no attribute 'normalize'